In [1]:
# ==========================================================
# EfficientNetV2-S | CIFAR-10 | 4 Epochs | No Hyperparameter Tuning | No Seeds
# ==========================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


In [2]:
# --------------------------
# Device
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# --------------------------
# Transformations & Dataloaders
# --------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:04<00:00, 39.4MB/s]


In [4]:
# --------------------------
# Model Definition 
# --------------------------
class EfficientNetV2S_Model(nn.Module):
    def __init__(self):
        super(EfficientNetV2S_Model, self).__init__()
        self.model = torchvision.models.efficientnet_v2_s(weights="IMAGENET1K_V1")

        
        for param in self.model.features.parameters():
            param.requires_grad = False

        # Replace classifier for CIFAR-10
        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, 10)

    def forward(self, x):
        return self.model(x)

In [5]:
# --------------------------
# Training Function
# --------------------------
def train_model(model, trainloader, criterion, optimizer, epochs=4):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {total_loss/len(trainloader):.4f}")

In [6]:
# --------------------------
# Evaluation Function
# --------------------------
def evaluate_model(model, testloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    return acc, prec, rec, f1

In [7]:
# --------------------------
# Main Run
# --------------------------
model = EfficientNetV2S_Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  

train_model(model, trainloader, criterion, optimizer, epochs=4)
acc, prec, rec, f1 = evaluate_model(model, testloader)

print("\n==============================")
print("📊 Evaluation Metrics")
print("==============================")
print(f"Accuracy:  {acc*100:.2f}%")
print(f"Precision: {prec*100:.2f}%")
print(f"Recall:    {rec*100:.2f}%")
print(f"F1-Score:  {f1*100:.2f}%")

Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


100%|██████████| 82.7M/82.7M [00:00<00:00, 172MB/s]


Epoch [1/4] - Loss: 1.0188
Epoch [2/4] - Loss: 0.8498
Epoch [3/4] - Loss: 0.8293
Epoch [4/4] - Loss: 0.8244

📊 Evaluation Metrics
Accuracy:  75.58%
Precision: 75.52%
Recall:    75.58%
F1-Score:  75.48%
